## Criando diretório gold

In [0]:
SILVER_PATH = "/Volumes/workspace/default/delta/silver"
GOLD_PATH = "/Volumes/workspace/default/delta/gold"
# Criar diretórios se não existirem
dbutils.fs.mkdirs(GOLD_PATH)

## Consultado os arquivos da camada silver

In [0]:
dbutils.fs.ls(f"{SILVER_PATH}/")